In [42]:
import pandas as pd

In [43]:
giay_nam = pd.read_csv('D:\TIKI\giay-nam.csv')
giay_nu = pd.read_csv('D:\TIKI\giay-nu.csv')
giay_nam = giay_nam.drop(columns=['Unnamed: 0'])
giay_nu = giay_nu.drop(columns=['Unnamed: 0'])

In [44]:
giay_nam_id = pd.read_csv('giay-nam-id.csv',usecols=['product_id','category_name'])
giay_nu_id = pd.read_csv('giay-nu-id.csv',usecols=['product_id','category_name'])
giay_nam_id = giay_nam_id.drop_duplicates()
giay_nu_id = giay_nu_id.drop_duplicates()

In [45]:
giay_nam = giay_nam.merge(giay_nam_id,left_on='id',right_on='product_id',how='left')
giay_nu = giay_nu.merge(giay_nu_id,left_on='id',right_on='product_id',how='left')
giay_nam = giay_nam.drop(columns=['product_id'])
giay_nu = giay_nu.drop(columns=['product_id'])

In [46]:
#I add tag 'man' / 'woman' into category_name to ensure that there are no identical category name in both table when concatenated later
giay_nam['category_name'] = 'MAN-'+giay_nam['category_name']
giay_nam['root_category'] = 'giay-nam'

giay_nu['category_name'] = 'WOMAN-'+giay_nu['category_name']
giay_nu['root_category'] = 'giay-nu'

In [51]:
concat_df = pd.concat([giay_nam,giay_nu])

In [52]:
concat_df

,id,sku,name,discription,original_price,list_price,price,alltime_quantity_sold,quantity_sold,inventory_status,...,review_count,rating_average,favourite_count,ASA_cashback,pay_later,current_seller,date_created,video_url,category_name,root_category
0,103682599,7076861254597,"D4 Dép Tổ Ong Đi Mưa, Đi Trong Nhà Nam Nữ Chất...","Dép Tổ Ong Đi Mưa, Đi Trong Nhà Nam Nữ ...",9900,9900,9900,95,95.0,available,...,7,3.3,0,"Thưởng 1,43 ASA (≈ 306đ)",NaN,Rumyh Fashion Style,743,NaN,MAN-dep-nam-quai-ngang,giay-nam
1,176818958,6576642454934,DÉP TỔ ONG ĐEN NHỰA CAO SU DẺO LOẠI ĐẸP ĐỦ SIZ...,LƯU Ý : KHÁCH MUA TỪ 5 ĐÔI TRỞ LÊN SHOP...,10000,10000,10000,17,17.0,available,...,0,0.0,0,"Thưởng 0,05 ASA (≈ 10đ)",NaN,Tổng kho đồ gia dụng QL,419,NaN,MAN-dep-nam-quai-ngang,giay-nam
2,119214657,1966875153167,dép tổ ong huyền thoại full size,Dép Làm Chất liệu cao su tổng hợp giúp ...,9900,9900,9900,81,81.0,available,...,9,3.2,0,"Thưởng 0,05 ASA (≈ 10đ)",NaN,sumo98 store,666,NaN,MAN-dep-nam-quai-ngang,giay-nam
3,164789853,3690478835294,Dép tổ ong đi trong nhà dày mỏng các loại,THÔNG TIN SẢN PHẨM DÉP TỔ ONG CÁC LOẠ...,24999,24999,24999,{},NaN,available,...,0,0.0,0,"Thưởng 0,12 ASA (≈ 26đ)",NaN,Giày dép Trường Hằng,478,NaN,MAN-dep-nam-quai-ngang,giay-nam
4,208440729,9890883168793,"Dép xốp QUAI NGANG đi trong nhà/Khách sạn/Spa,...","Dép xốp quai ngang, đi trong nhà/Khách ...",14200,14200,14200,{},NaN,available,...,0,0.0,0,"Thưởng 0,07 ASA (≈ 15đ)",NaN,TT8695,191,NaN,MAN-dep-nam-quai-ngang,giay-nam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25027,25388902,3397953659148,Dép nhựa đi mưa đính nơ trong suốt siêu cute c...,"Form chuẩnThích hợp đi mùa đông, mùa hèSize 35...",67500,67500,67500,188,188.0,available,...,25,4.5,0,"Thưởng 0,34 ASA (≈ 71đ)",NaN,Jay_shop,1412,NaN,WOMAN-dep-quai-ngang,giay-nu
25028,191427347,7534403715444,Dép bánh mì quai ngang ly trà sữa siêu cưng,Dép quai ngang ly trà sữa siêu cưng Hàng Q...,79000,79000,79000,{},NaN,available,...,0,0.0,0,"Thưởng 0,40 ASA (≈ 83đ)",NaN,Giày Dép Thùy Phan,337,NaN,WOMAN-dep-quai-ngang,giay-nu
25029,112500211,5467110952991,Dép bánh mì quai ngang nữ mẫu mới Hottrend 202...,Dép bánh mì quai ngang nữ mẫu mới Hottrend 202...,68000,68000,68000,7,7.0,available,...,1,5.0,0,"Thưởng 9,79 ASA (≈ 2.075đ)",NaN,HƯƠNG GIÀY,706,NaN,WOMAN-dep-quai-ngang,giay-nu
25030,193329161,3793215273206,Dép bánh mì in nổi hình Gấu cao 4cm nữ quai ng...,(*) DÉP BÁNH MÌ QUAI NGANG GẤU TEDDY SIÊU NH...,59000,59000,59000,{},NaN,available,...,0,0.0,0,"Thưởng 0,29 ASA (≈ 62đ)",NaN,IDÉP Giày Dép Nam Nữ,243,NaN,WOMAN-dep-quai-ngang,giay-nu
